In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import scipy.io as sio
from dataclasses import dataclass
from typing import List, Tuple
import os
from dotenv import load_dotenv
load_dotenv()
import tidy3d as td
from tidy3d import web
import numpy as np
from pathlib import Path
from stl import mesh
import matplotlib.pyplot as plt

In [2]:
import sys
import os

# Assuming /AutomationModule is in the root directory of your project
sys.path.append(os.path.abspath(r'H:\phd stuff\tidy3d'))

from AutomationModule import * 

import AutomationModule as AM

In [3]:
def get_coords(multiplication_factor,t_slab_x,t_slab_y,t_slab_z):
    x_size,y_size,z_size = t_slab_x/multiplication_factor,t_slab_y/multiplication_factor,t_slab_z
    coordinates_slabs=[]
    for i in range(multiplication_factor):
        for j in range(multiplication_factor):
            center_x = (i - ( multiplication_factor/ 2) + 0.5) * x_size
            center_y = (j - ( multiplication_factor/ 2) + 0.5) * y_size
            center_z = 0  # All cubes are centered on the z=0 plane
            coord_item = {
                    "X": (center_x - x_size/2, center_x + x_size/2),
                    "Y": (center_y - y_size/2, center_y + y_size/2),
                    "Z": (-z_size/2, z_size/2),
                    "center": (center_x, center_y, center_z)
                    }
            
            coordinates_slabs+=[coord_item]

    return coordinates_slabs

In [4]:
# Function to create a solid cylinder with specified center coordinates
def create_cylinder_from_ends(top_center, bottom_center, radius):
    # Calculate height of the cylinder
    height = np.linalg.norm(np.array(top_center) - np.array(bottom_center))
    bottom_center=bottom_center

    # Calculate the vector direction of the cylinder
    axis_direction = np.array(top_center) - np.array(bottom_center)
    axis_direction /= np.linalg.norm(axis_direction) #Unitary vector to calculate rotation angle 

    # Calculate the rotation matrix to align cylinder with the given axis direction
    z_axis = np.array([0, 0, 1])
    rotation_axis = np.cross(z_axis, axis_direction)
    rotation_angle = np.arccos(np.dot(z_axis, axis_direction))
    rotation_matrix = mesh.Mesh.rotation_matrix(rotation_axis, rotation_angle)
    # Create a 4x4 identity matrix
    matrix_4x4 = np.eye(4)

    # Insert the 3x3 matrix into the top-left corner of the new matrix
    matrix_4x4[:3, :3] = np.copy(rotation_matrix)

    # Set the fourth column and fourth row for homogeneous transformation
    # The typical homogeneous transformation uses [0, 0, 0, 1] for the last row and column
    matrix_4x4[3, :3] = [0, 0, 0]  # Fourth row
    matrix_4x4[:3, 3] = [0, 0, 0]  # Fourth column
    matrix_4x4[3, 3] = 1           # Bottom-right corner remains 1

    
   
    cylinder_center = tuple((np.array(top_center) + np.array(bottom_center))/2)
    trans = td.Transformed.translation(cylinder_center[0],cylinder_center[1],cylinder_center[2])
    transformed = td.Transformed(geometry=td.Cylinder(center=(0,0,0), radius=radius, length=height),transform=trans@matrix_4x4)

    return transformed

In [5]:
tidy3dAPI = os.environ["API_TIDY3D_KEY"]

In [6]:
a = 5/3
run=False

In [7]:
# lambdas = a/np.array([0.21,0.23]) 
lambdas = a/np.array([0.435,0.455]) 
folder_path = r"H:\phd stuff\tidy3d\structures\08_13_2024 Luis H5 18.01a\sample 2"
project_name = "09_04_2024 RCP Tidy Structures Beam Spreading"
# project_name = "08_28_2024 RCP Tidy Structures Beam Spreading"
postprocess_results = []
runtime_ps = 24e-12
min_steps_per_lambda = 16
multiplication_factor=6
for direction in ["z"]: 
    for filename in os.listdir(folder_path):
    #  for perm in [50]:
     for perm in [15]:
        print(filename)
        sim_name = f"RCP perm {perm} runtime {runtime_ps}ps nu {a/lambdas[0]} - {a/lambdas[1]} - {multiplication_factor*18}x{multiplication_factor*18}x18a"

        if not (Path(filename).suffix==".h5" or Path(filename).suffix==".stl"):
            continue 
        if os.path.isfile(os.path.join(folder_path, filename)):
            file=os.path.join(folder_path, filename)
            structure_1 = AM.loadAndRunStructure(key = tidy3dAPI, file_path=file
                                            ,direction=direction, lambda_range=lambdas,
                                            box_size=18*a,runtime_ps=runtime_ps,min_steps_per_lambda=min_steps_per_lambda, permittivity=perm,
                                           scaling=1,shuoff_condtion=1e-20, verbose=True, 
                                           monitors=["flux"], freqs=100, 
                                           cut_condition=1.0, source="tight", absorbers=200, sim_name=sim_name, use_permittivity=False, tight_percentage=0.05,
                                           ref_only=True, multiplicate_size=True,multiplication_factor=multiplication_factor,width=0.2
                                           )
           

            


02_Sample_600_res_L18.01_28ff_eps.h5
Configured successfully.


In [8]:
file_path =  r"H:\phd stuff\tidy3d\structures\End2EndFiles\RCP 18.01 End Points\1_sample_L18_lines_cut.dat"
data = []
with open(file_path, 'r') as file:
    for line in file: 
        row = [float(value) for value in line.split()]
        data += [np.array(row)]

data = np.array(data)

tops_centered = (data[:,:3]-0.5)*structure_1.t_slab_z
bottoms_centered = (data[:,3:]-0.5)*structure_1.t_slab_z

radius = 0.25


cyl_group = []



coords = get_coords(structure_1.multiplication_factor,structure_1.t_slab_x,structure_1.t_slab_y,structure_1.t_slab_z)


for k,coord in enumerate(coords):
    tops = np.copy(tops_centered)
    bottoms = np.copy(bottoms_centered)
    tops[:,0]+=coord["center"][0]
    tops[:,1]+=coord["center"][1]
    tops[:,2]+=coord["center"][2]
    bottoms[:,0]+=coord["center"][0]
    bottoms[:,1]+=coord["center"][1]
    bottoms[:,2]+=coord["center"][2]
    for cil,item in enumerate(tops):
         cyl_group.append(create_cylinder_from_ends(tops[cil], bottoms[cil], radius))

    
   


In [9]:
sim = structure_1.sim


In [10]:
structure_1.permittivity_value

15

In [11]:
medium=td.Medium(permittivity=structure_1.permittivity_value)
structure = td.Structure(geometry=td.GeometryGroup(geometries=cyl_group),  medium=medium)
cube = td.Structure(geometry= td.Box(
                        center=(0,  0 ,0),
                        size=(
                                structure_1.t_slab_x, 
                                structure_1.t_slab_y, 
                                structure_1.t_slab_z
                              ),
                    ),medium=td.Medium(permittivity=1))



In [12]:
boundaries= td.BoundarySpec(
            x=td.Boundary(plus=td.Absorber(num_layers=150),minus=td.Absorber(num_layers=150)),
            y=td.Boundary(plus=td.Absorber(num_layers=150),minus=td.Absorber(num_layers=150)),
            z=td.Boundary(plus=td.Absorber(num_layers=150),minus=td.Absorber(num_layers=150)),
        )
Lx, Ly, Lz =structure_1.Lx+4,structure_1.Ly+4,structure_1.Lz+10


time_monitorFieldLateral = td.FieldTimeMonitor(
                center = (0,0,0),
                size = (0,Ly,Lz),
                    start=0,
                    stop=structure_1.t_stop,
                    interval=200,
                    fields=["Ex", "Ey", "Ez"],
                    name="time_monitorFieldLateral",
                )



time_monitorT = td.FluxTimeMonitor(
                    center=[
                        (structure_1.Lx - structure_1.spacing)*0.5 if structure_1.direction == "x" else 0, 
                        (structure_1.Ly - structure_1.spacing)*0.5 if structure_1.direction == "y" else 0, 
                        (structure_1.Lz - structure_1.spacing)*0.5 if structure_1.direction == "z" else 0


                        ],
                    size=[
                        td.inf,td.inf,0
                        ],
                    interval = 200,
                    name="time_monitorT",

                )

In [13]:
eps_eff  = ((2+structure_1.permittivity_value )+ 2*0.3*(structure_1.permittivity_value-1))/((2+structure_1.permittivity_value)-0.3*(structure_1.permittivity_value-1))
medium_eff=td.Medium(permittivity=eps_eff)
box_in = td.Box(center=[0, 0,-(Lz/2+structure_1.t_slab_z/2)/2], size=[td.inf, td.inf, Lz / 2-structure_1.t_slab_z/2])
box_out = td.Box(center=[0, 0,(Lz/2+structure_1.t_slab_z/2)/2], size=[td.inf, td.inf, Lz / 2-structure_1.t_slab_z/2 ])
box_up = td.Box(center=[0,(Ly/2+structure_1.t_slab_y/2)/2,0], size=[td.inf, Ly / 2-structure_1.t_slab_y/2 ,td.inf])
box_down = td.Box(center=[0, -(Ly/2+structure_1.t_slab_y/2)/2 ,0], size=[td.inf, Ly / 2-structure_1.t_slab_y/2, td.inf ])
struct_in = td.Structure(geometry=box_in, medium=medium_eff)
struct_out = td.Structure(geometry=box_out, medium=medium_eff)
struct_up = td.Structure(geometry=box_up, medium=medium_eff)
struct_down = td.Structure(geometry=box_down, medium=medium_eff)

In [14]:
sim = sim.copy(update={"structures":[structure,struct_in, struct_out,struct_up,struct_down],'boundary_spec':boundaries,'size':[Lx,Ly,Lz],"monitors":[time_monitorT,time_monitorFieldLateral]})


17:53:32 W. Europe Daylight Time WARNING: Structure at 'structures[1]' has      
                                 bounds that extend exactly to simulation edges.
                                 This can cause unexpected behavior. If         
                                 intending to extend the structure to infinity  
                                 along one dimension, use td.inf as a size      
                                 variable instead to make this explicit.        

                                 WARNING: Suppressed 3 WARNING messages.        

In [15]:
# sim.plot_3d()

In [16]:
# fig, ax = plt.subplots(1, tight_layout=True, figsize=(16, 8))
# freqs_plot = (structure_1.freq_range[0], structure_1.freq_range[1])
# sim.plot_eps(x=0, freq=freqs_plot[0], ax=ax)
# plt.show()

# fig, ax = plt.subplots(1, tight_layout=True, figsize=(16, 8))
# freqs_plot = (structure_1.freq_range[0], structure_1.freq_range[1])
# sim.plot_eps(z=0, freq=freqs_plot[0], ax=ax)
# plt.show()


In [17]:
id =web.upload(sim,task_name=sim_name, verbose=True)
web.estimate_cost(task_id = id)

17:55:36 W. Europe Daylight Time WARNING: Monitor 'time_monitorFieldLateral'    
                                 estimated storage is 23.07GB. Consider making  
                                 it smaller, using fewer frequencies, or spatial
                                 or temporal downsampling using 'interval_space'
                                 and 'interval', respectively.                  

17:55:40 W. Europe Daylight Time Created task 'RCP perm 15 runtime 2.4e-11ps nu 
                                 0.435 - 0.455 - 108x108x18a' with task_id      
                                 'fdve-1b25843d-6e83-4044-871f-0156265f23e5' and
                                 task_type 'FDTD'.

                                 View task using web UI at                      
                                 ]8;id=841970;https://tidy3d.simulation.cloud/workbench?taskId=fdve-1b25843d-6e83-4044-871f-0156265f23e5\'https://tidy3d.simulation.cloud/workbench?]8;;\]8;id=930934;https://tidy3d.simulation.cloud/workbench?taskId=fdve-1b25843d-6e83-4044-871f-0156265f23e5\task]8;;\
                                 ]8;id=930934;https://tidy3d.simulation.cloud/workbench?taskId=fdve-1b25843d-6e83-4044-871f-0156265f23e5\Id]8;;\]8;id=841970;https://tidy3d.simulation.cloud/workbench?taskId=fdve-1b25843d-6e83-4044-871f-0156265f23e5\=]8;;\]8;id=667730;https://tidy3d.simulation.cloud/workbench?taskId=fdve-1b25843d-6e83-4044-871f-0156265f23e5\fdve]8;;\]8;id=841970;https://tidy3d.simulation.cloud/workbench?taskId=fdve-1b25843d-6e83-4044-871f-0156265f23e5\-1b25843d-6e83-4044-871f-0156265f23e5']8;;\.

Output()

18:38:22 W. Europe Daylight Time Maximum FlexCredit cost: 225.601. Minimum cost 
                                 depends on task execution details. Use         
                                 'web.real_cost(task_id)' to get the billed     
                                 FlexCredit cost after a simulation run.

225.60094009209098

In [18]:
if run:
        id0 = ''
        # print("running ref...")
        # id0 = web.upload(sim.copy(update={"structures":[]}), folder_name=project_name,task_name=sim_name+'_0', verbose=True)
        # web.start(task_id = id0)
        # web.monitor(task_id=id0,verbose=True)

        id =web.upload(sim, folder_name=project_name,task_name=sim_name, verbose=True)
        web.start(task_id = id)
        web.monitor(task_id=id,verbose=True)
    
        ids = (id0 if id0 else '') +'\n' + id
        incidence_folder = "z_incidence"
        file_path = rf"H:\phd stuff\tidy3d\data/{project_name}/{incidence_folder}/{sim_name}.txt"
        # Check if the folder exists
        if not os.path.exists( rf"H:\phd stuff\tidy3d\data/{project_name}/{incidence_folder}"):
            os.makedirs(rf"H:\phd stuff\tidy3d\data/{project_name}/{incidence_folder}")
            print(f"Folder '{project_name}/{incidence_folder}' created successfully.")

        # Open file in write mode
        with open(file_path, "w") as file:
            # Write the string to the file
            file.write(ids)